In [ ]:
#@title 🚀 Ailo Colab Miner - One Click Mining!
#@markdown ### Instructions:
#@markdown 1. Runtime > Change runtime type > **T4 GPU**
#@markdown 2. Enter your wallet address below
#@markdown 3. Press **▶️ Play** - that's it!
#@markdown ---

WALLET = ""  #@param {type:"string"}

# ===== DO NOT EDIT BELOW =====
import subprocess, sys
subprocess.run([sys.executable, '-m', 'pip', 'install', 'torch', '--index-url', 'https://download.pytorch.org/whl/cu118', '-q'], capture_output=True)
subprocess.run([sys.executable, '-m', 'pip', 'install', 'aiohttp', 'requests', '-q'], capture_output=True)

import torch, torch.nn as nn, numpy as np, requests, aiohttp, asyncio, time, base64, gzip, gc, os

API, SERVER, VER = "https://ailo.site/api", "https://ailo.site", "1.8.0-colab"
MAX_LOSS_FOR_SUBMIT = 2.8  # STRICT: Don't submit if loss is higher than global model

if not torch.cuda.is_available():
    print("❌ GPU not enabled! Go to: Runtime > Change runtime type > T4 GPU")
    raise SystemExit

if len(WALLET) < 40:
    print("❌ Enter your wallet address above! Get one at: https://ailo.site/wallet.html")
    raise SystemExit

print(f"✅ GPU: {torch.cuda.get_device_name(0)} | VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB")
print(f"✅ Wallet: {WALLET[:12]}...")

class AILO1B(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(50257, 1600)
        self.pos = nn.Parameter(torch.zeros(1, 512, 1600))
        self.tf = nn.TransformerEncoder(nn.TransformerEncoderLayer(1600, 25, 6400, 0.1, batch_first=True), 24)
        self.out = nn.Linear(1600, 50257)
    def forward(self, x):
        return self.out(self.tf(self.emb(x) * 40 + self.pos[:, :x.size(1)]))

class Trainer:
    def __init__(self):
        self.dev = torch.device('cuda')
        self.m = None; self.opt = None; self.steps = 0; self.acc = 0
        self.best_loss = float('inf'); self.local_version = 0
        self.saved_grads = None
        self.synced = False  # CRITICAL: Track if we synced with global model
    
    def init(self):
        print("🧠 Loading AILO-1B (899M params)...")
        gc.collect(); torch.cuda.empty_cache()
        self.m = AILO1B().half().to(self.dev)
        print(f"   ✅ Ready! GPU Memory: {torch.cuda.memory_allocated()/1e9:.1f}GB")
        self.opt = torch.optim.SGD(self.m.parameters(), lr=0.001, momentum=0.9)
        self.crit = nn.CrossEntropyLoss()
    
    def sync_with_global(self):
        try:
            print("🔄 Syncing with global model...")
            r = requests.get(f"{SERVER}/api/model/weights", timeout=30)
            if r.status_code != 200: 
                print("   ⚠️ No global model available")
                return False
            data = r.json()
            if data.get('hasCheckpoint', False):
                server_loss = data.get('bestCheckpointLoss', 999)
                print(f"   📥 Downloading checkpoint (loss: {server_loss})...")
                cr = requests.get(f"{SERVER}/api/model/checkpoint", timeout=60)
                if cr.status_code == 200:
                    w = np.frombuffer(cr.content, dtype=np.float16).astype(np.float32)
                    with torch.no_grad():
                        exp = self.m.out.weight.numel()
                        if len(w) >= exp:
                            self.m.out.weight.data = torch.from_numpy(w[:exp]).reshape_as(self.m.out.weight).half().to(self.dev)
                            self.synced = True
                            self.best_loss = float(server_loss) if server_loss != 'N/A' else 2.8
                            print(f"   ✅ Loaded {exp:,} weights (target loss: {self.best_loss:.4f})")
                    return True
            return False
        except Exception as e:
            print(f"   ⚠️ Sync failed: {e}")
            return False
    
    def upload_checkpoint(self, wallet, loss, epoch):
        try:
            with torch.no_grad():
                w = self.m.out.weight.data.cpu().half().numpy().tobytes()
                w64 = base64.b64encode(w).decode()
            r = requests.post(f"{SERVER}/api/model/checkpoint", json={'wallet': wallet, 'weights': w64, 'loss': loss, 'epoch': epoch}, timeout=60)
            if r.status_code == 200 and r.json().get('success'):
                print(f"   📦 Checkpoint uploaded!")
        except: pass
    
    def batch(self, w):
        try:
            r = requests.get(f"{API}/cuda/training-data", params={'batchSize': 1, 'wallet': w}, timeout=10)
            if r.ok:
                t = r.json().get('articles', [''])[0]
                if len(t) >= 33:  # CRITICAL: Only use valid data
                    tk = [ord(c) % 50257 for c in t[:33]]
                    return torch.tensor([tk[:32]]), torch.tensor([tk[1:33]])
        except: pass
        # NO FALLBACK TO RANDOM - return None to skip this batch
        return None, None
    
    def step(self, x, y):
        if x is None: return None  # Skip if no valid data
        self.m.train()
        x, y = x.to(self.dev), y.to(self.dev)
        with torch.cuda.amp.autocast():
            loss = self.crit(self.m(x).view(-1, 50257), y.view(-1)) / 16
        loss.backward()
        self.acc += 1
        if self.acc >= 16:
            nn.utils.clip_grad_norm_(self.m.parameters(), 1.0)
            g = self.m.out.weight.grad
            if g is not None:
                self.saved_grads = g.clone().cpu()
            self.opt.step(); self.opt.zero_grad(set_to_none=True); self.acc = 0
        self.steps += 1
        l = loss.item() * 16
        if l < self.best_loss and not np.isnan(l): self.best_loss = l
        if self.steps % 50 == 0: gc.collect(); torch.cuda.empty_cache()
        return l
    
    def grads(self):
        g = self.saved_grads
        if g is None: 
            print("   ⚠️ No gradients yet")
            return None
        gc.collect()
        flat = g.float().flatten()
        sample = flat[::10]
        comp = gzip.compress(sample.half().numpy().tobytes(), 4)
        return base64.b64encode(comp).decode()

async def hb(w, tps):
    try:
        async with aiohttp.ClientSession() as s:
            await s.post(f"{API}/cuda/register", json={'wallet': w, 'clientVersion': VER, 'deviceInfo': {'gpu_name': 'T4-Colab', 'vram_gb': 15, 'hashrate': tps}})
    except: pass

async def submit(w, g, s, l):
    try:
        async with aiohttp.ClientSession() as ss:
            async with ss.post(f"{API}/cuda/submit", json={'wallet': w, 'gradients': g, 'epoch': s, 'loss': l, 'gpu': 'T4-Colab'}) as r:
                if r.status == 200: return (await r.json()).get('reward', 0)
    except: pass
    return 0

async def run():
    print("=" * 50)
    print(f"  🚀 AILO-1B Colab Miner v{VER}")
    print("=" * 50)
    
    await hb(WALLET, 0)
    t = Trainer(); t.init()
    
    # CRITICAL: Must sync before training!
    if not t.sync_with_global():
        print("\n⚠️ Could not sync with global model!")
        print("   Will train but NOT submit gradients until synced.")
    
    print("\n⛏️ MINING STARTED!")
    print(f"📊 Dashboard: https://ailo.site/dashboard.html?wallet={WALLET[:12]}\n")
    
    rew, last_s, last_h, tps = 0.0, time.time(), time.time(), 0.0
    skipped_batches = 0
    
    try:
        while True:
            t0 = time.time()
            x, y = t.batch(WALLET)
            if x is None:
                skipped_batches += 1
                if skipped_batches % 10 == 0:
                    print(f"⏭️ Skipped {skipped_batches} batches (no valid data)")
                await asyncio.sleep(0.5)
                continue
            
            loss = t.step(x, y)
            if loss is None: continue
            
            tps = 32 / (time.time() - t0 + 0.001)
            
            if time.time() - last_h >= 10: await hb(WALLET, tps); last_h = time.time()
            if t.steps % 25 == 0: print(f"Step {t.steps} | Loss: {loss:.4f} | Best: {t.best_loss:.4f} | {tps:.0f} tok/s | 💰 {rew:.4f} ALC")
            
            if time.time() - last_s >= 300:
                gc.collect(); torch.cuda.empty_cache()
                g = t.grads()
                
                # CRITICAL: Only submit if synced AND loss is good
                if g and t.synced and t.best_loss < MAX_LOSS_FOR_SUBMIT and not np.isnan(t.best_loss):
                    print(f"\n📤 Submitting gradients (loss: {t.best_loss:.4f})...")
                    r = await submit(WALLET, g, t.steps, t.best_loss); rew += r
                    print(f"💰 +{r:.4f} ALC (Total: {rew:.4f})")
                    if t.best_loss < 2.5: t.upload_checkpoint(WALLET, t.best_loss, t.steps)
                elif not t.synced:
                    print(f"\n⏳ Not synced with global model - training locally...")
                elif g:
                    print(f"\n⏳ Loss too high ({t.best_loss:.4f} > {MAX_LOSS_FOR_SUBMIT}) - training more...")
                else:
                    print("\n⏭️ No gradients yet")
                
                t.sync_with_global()
                last_s = time.time()
                print()
                
    except KeyboardInterrupt: print(f"\n⏹️ Stopped. Total earned: {rew:.4f} ALC")

await run()